In [65]:
import pandas as pd
import numpy as np

df = pd.read_csv("coffee_cognition_realistic_processed.csv")

In [66]:
df.head()
# df.columns

,Participant_ID,Age,Sleep_Hours,Sleep_Quality_Score,Stress_Level,Physical_Activity_Level,Caffeine_mg,Stroop_Reaction_Time_ms,PVT_Reaction_Time_ms,N_Back_Accuracy,...,Brewing_Method_Espresso,Brewing_Method_French Press,Brewing_Method_Pour Over,Time_of_Day_Evening,Time_of_Day_Morning,Gender_Male,Caffeine_x_SleepQuality,Caffeine_x_Stress,Caffeine_x_Time_of_Day,Caffeine_sq
0,P2000,0.956522,0.852445,-0.003767,-0.707241,-0.781066,0.924902,588.863994,180.000000,0.685505,...,False,False,False,True,False,True,-0.003485,-0.654128,0.924902,0.855443
1,P2001,-0.913043,1.581432,-0.182437,-1.061402,-0.992460,0.255743,789.798695,227.346446,0.643084,...,False,True,False,False,False,False,-0.046657,-0.271446,0.000000,0.065404
2,P2002,0.217391,0.402326,0.651556,-0.502150,1.060463,0.937205,627.187730,260.779800,0.834411,...,False,False,False,False,True,True,0.610642,-0.470617,0.000000,0.878353
3,P2003,0.478261,-0.455266,-0.031149,1.551318,-0.536018,0.367670,690.477498,251.770710,0.759299,...,False,False,False,False,True,False,-0.011452,0.570374,0.000000,0.135182
4,P2004,0.652174,0.049296,0.979470,0.735099,-0.318509,-0.563629,677.910485,309.438359,0.697677,...,True,False,False,False,True,True,-0.552058,-0.414323,-0.000000,0.317678


In [67]:
targets = [
    "Stroop_Reaction_Time_ms",
    "N_Back_Accuracy",
    "PVT_Reaction_Time_ms",
    "Focus_Level"
]

x = df.drop(columns=targets)
x = x.drop(columns="Participant_ID")

y_stroop = df["Stroop_Reaction_Time_ms"]
y_nback = df["N_Back_Accuracy"]
y_pvt = df["PVT_Reaction_Time_ms"]
y_focus = df["Focus_Level"]

In [68]:
x.head()

,Age,Sleep_Hours,Sleep_Quality_Score,Stress_Level,Physical_Activity_Level,Caffeine_mg,Brewing_Method_Drip,Brewing_Method_Espresso,Brewing_Method_French Press,Brewing_Method_Pour Over,Time_of_Day_Evening,Time_of_Day_Morning,Gender_Male,Caffeine_x_SleepQuality,Caffeine_x_Stress,Caffeine_x_Time_of_Day,Caffeine_sq
0,0.956522,0.852445,-0.003767,-0.707241,-0.781066,0.924902,True,False,False,False,True,False,True,-0.003485,-0.654128,0.924902,0.855443
1,-0.913043,1.581432,-0.182437,-1.061402,-0.992460,0.255743,False,False,True,False,False,False,False,-0.046657,-0.271446,0.000000,0.065404
2,0.217391,0.402326,0.651556,-0.502150,1.060463,0.937205,False,False,False,False,False,True,True,0.610642,-0.470617,0.000000,0.878353
3,0.478261,-0.455266,-0.031149,1.551318,-0.536018,0.367670,True,False,False,False,False,True,False,-0.011452,0.570374,0.000000,0.135182
4,0.652174,0.049296,0.979470,0.735099,-0.318509,-0.563629,False,True,False,False,False,True,True,-0.552058,-0.414323,-0.000000,0.317678


In [69]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 

In [80]:
def evaluate_models(models, X, targets, n_splits=5):
    """
    Evaluate multiple regression models on multiple targets using cross-validation
    
    Parameters:
        models (dict): {model_name: model_instance}
        X (DataFrame): Feature matrix
        targets (dict): {target_name: y_series}
        n_splits (int): Number of CV folds
    
    Returns:
        DataFrame: Comparison table with RMSE, MAE, R2
    """
    
    results = []

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for model_name, model in models.items():
        for target_name, y in targets.items():
            
            rmse_list, mae_list, r2_list = [], [], []

            for train_idx, test_idx in kf.split(X):
                X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

                model.fit(X_train, y_train)
                preds = model.predict(X_test)

                rmse_list.append(
                    np.sqrt(mean_squared_error(y_test, preds))
                )
                mae_list.append(
                    mean_absolute_error(y_test, preds)
                )
                r2_list.append(
                    r2_score(y_test, preds)
                )

            results.append({
                "Model": model_name,
                "Target": target_name,
                "RMSE": np.mean(rmse_list),
                "MAE": np.mean(mae_list),
                "R2": np.mean(r2_list)
            })

    return pd.DataFrame(results)


In [81]:
targets = {
    "Stroop_RT": y_stroop,
    "PVT_RT": y_pvt,
    "NBack_Accuracy": y_nback,
    "Focus_Level": y_focus
}


In [91]:
pip install xgboost lightgbm


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
    --------------------------------------- 1.0/72.0 MB 5.8 MB/s eta 0:00:13
   - -------------------------------------- 1.8/72.0 MB 5.4 MB/s eta 0:00:14
   - -------------------------------------- 2.6/72.0 MB 4.7 MB/s eta 0:00:15
   - -------------------------------------- 3.4/72.0 MB 4.3 MB/s eta 0:00:16
   -- ------------------------------------- 4.2/72.0 MB 4.3 MB/s eta 0:00:16
   -- ------------------------------------- 5.2/72.0 MB 4.4 MB/s eta 0:00:16
   --- ------------------------------------ 6.6/72.0 MB 4.6 MB/s eta 0:00:15
   ---- ----------------------------------- 7.3/72.0 MB 4.7 MB/s eta 0:00:14
   ---- ----------------------------------- 8.1/72.0 MB 4.6 MB/s eta 0:00:14
   ---- ----------------------------------- 8.9/72.0 MB 4.3 MB/s eta 0:00:15
   ---- ----------------------------------- 8.9/72.0 MB 4.3 MB/s eta 0:00:15
   ----- ---------------------------------- 9.4/72.0 MB 3.9 MB/s eta 0:00:17
   --

In [92]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import (
    GradientBoostingRegressor, 
    AdaBoostRegressor, 
    ExtraTreesRegressor,
    HistGradientBoostingRegressor,
    RandomForestRegressor
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
# Optional but highly recommended
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [93]:
models = {
    # Your existing models...
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(alpha=0.1),
    "Lasso": Lasso(alpha=0.001),
    "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=300, random_state=42),
    "SVR (RBF)": SVR(kernel="rbf", C=10, epsilon=0.1),
    
    # NEW: Gradient Boosting models (usually best for tabular data)
    "Gradient Boosting": GradientBoostingRegressor(
        n_estimators=200, 
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    ),
    "XGBoost": XGBRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=200,
        learning_rate=0.1,
        random_state=42,
        verbose=-1
    ),
    "HistGradientBoosting": HistGradientBoostingRegressor(
        max_iter=200,
        learning_rate=0.1,
        random_state=42
    ),
    
    # NEW: Other models worth trying
    "Extra Trees": ExtraTreesRegressor(n_estimators=200, random_state=42),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "MLP": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42),
    "AdaBoost": AdaBoostRegressor(n_estimators=100, random_state=42),
}

In [94]:
results_df = evaluate_models(models, x, targets)

results_df


c:\Users\STTL\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\STTL\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\STTL\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\STTL\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\STTL\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer

,Model,Target,RMSE,MAE,R2
0,Linear Regression,Stroop_RT,61.307813,48.183134,0.334283
1,Linear Regression,PVT_RT,30.400113,24.445184,0.442721
2,Linear Regression,NBack_Accuracy,0.069069,0.054527,0.537277
3,Linear Regression,Focus_Level,0.102124,0.078728,0.471551
4,Ridge,Stroop_RT,61.305523,48.180495,0.334335
5,Ridge,PVT_RT,30.398208,24.444105,0.442797
6,Ridge,NBack_Accuracy,0.069066,0.054525,0.537318
7,Ridge,Focus_Level,0.102121,0.078729,0.471586
8,Lasso,Stroop_RT,61.306408,48.181670,0.334315
9,Lasso,PVT_RT,30.398711,24.444199,0.442775


In [95]:
results_df.sort_values(by="R2", ascending=False)

,Model,Target,RMSE,MAE,R2
10,Lasso,NBack_Accuracy,0.068481,0.053932,0.544899
14,ElasticNet,NBack_Accuracy,0.068693,0.054177,0.542075
6,Ridge,NBack_Accuracy,0.069066,0.054525,0.537318
2,Linear Regression,NBack_Accuracy,0.069069,0.054527,0.537277
18,Random Forest,NBack_Accuracy,0.072728,0.057292,0.486504
42,Extra Trees,NBack_Accuracy,0.073058,0.057761,0.482236
11,Lasso,Focus_Level,0.101352,0.078259,0.479560
15,ElasticNet,Focus_Level,0.101648,0.078389,0.476422
7,Ridge,Focus_Level,0.102121,0.078729,0.471586
3,Linear Regression,Focus_Level,0.102124,0.078728,0.471551


In [96]:
best_models = results_df.loc[
    results_df.groupby("Target")["R2"].idxmax()
]

best_models


,Model,Target,RMSE,MAE,R2
11,Lasso,Focus_Level,0.101352,0.078259,0.479560
10,Lasso,NBack_Accuracy,0.068481,0.053932,0.544899
13,ElasticNet,PVT_RT,30.394222,24.441706,0.442956
12,ElasticNet,Stroop_RT,61.300804,48.175115,0.334442


In [58]:
target_model_map = {
    "Stroop_RT": Ridge(alpha=1.0),
    "PVT_RT": Ridge(alpha=1.0),
    "NBack_Accuracy": Lasso(alpha=0.001),
    "Focus_Level": Lasso(alpha=0.001)
}

In [60]:
from sklearn.base import clone

trained_models = {}

for target, model in target_model_map.items():
    m = clone(model)          # important: fresh model
    m.fit(x, targets[target]) # fit on full data
    trained_models[target] = m


In [61]:
def predict_all(X_new, trained_models):
    predictions = {}

    for target, model in trained_models.items():
        predictions[target] = model.predict(X_new)

    return predictions


In [62]:
import pandas as pd

def predict_all_df(X_new, trained_models):
    return pd.DataFrame({
        target: model.predict(X_new)
        for target, model in trained_models.items()
    })


In [64]:
pred_df = predict_all_df(x, trained_models)
print(pred_df.head())


    Stroop_RT      PVT_RT  NBack_Accuracy  Focus_Level
0  554.109171  229.902249        0.940370     0.884916
1  551.647570  215.609851        0.940317     0.888363
2  534.327914  213.858304        0.953837     0.877989
3  478.487105  221.239006        1.097826     1.073174
4  609.997954  277.375347        0.807711     0.716360
